##Logistic_Regression_From_Scratch

In [1]:
#Importing libraries
import numpy as np
import pandas as pd
import io
import matplotlib.pyplot as plt

In [2]:
datasets = pd.read_csv('BuyComputer.csv') 
print(datasets)

      User ID  Age  EstimatedSalary  Purchased
0    15624510   19            19000          0
1    15810944   35            20000          0
2    15668575   26            43000          0
3    15603246   27            57000          0
4    15804002   19            76000          0
..        ...  ...              ...        ...
395  15691863   46            41000          1
396  15706071   51            23000          1
397  15654296   50            20000          1
398  15755018   36            33000          0
399  15594041   49            36000          1

[400 rows x 4 columns]


In [3]:
datasets.drop(columns=['User ID',],axis=1,inplace=True)
datasets.head()

,Age,EstimatedSalary,Purchased
0,19,19000,0
1,35,20000,0
2,26,43000,0
3,27,57000,0
4,19,76000,0


In [4]:
# newdata = datasets.iloc[:,0:2]
# newtarget = datasets.iloc[:,2:3]
# print(newdata)
# print(newtarget)
targets = datasets.loc[:, datasets.columns == "Purchased"].values
features = datasets.loc[:, datasets.columns != "Purchased"].values

In [5]:
#import the necessary module
from sklearn.model_selection import train_test_split
#split data set into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(features,targets, test_size = 0.2, random_state = 113)

In [6]:
# Sacaling data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
#print(X_train)
#print(X_test)

In [7]:
class LogisticRegression:
  
  def initialize(self, x):
    weights = np.zeros((np.shape(x)[1] + 1, 1))
    x = np.c_[np.ones((np.shape(x)[0], 1)), x]
    return weights, x

  def sigmoid(self, z):
    sig = 1 / (1 + np.exp(-z))
    return sig

  def fit(self, x, y, alpha = 0.001, iter = 3000):
    weights, x = self.initialize(x)

    def cost(theta):
      z = np.dot(x, theta)
      cost0 = y.T.dot(np.log(self.sigmoid(z)))
      cost1 = (1-y).T.dot(np.log(1 - self.sigmoid(z)))
      cost = -((cost1 + cost0)) / len(y)
      return cost
    
    cost_list = np.zeros(iter,)
    for i in range(iter):
      weights = weights - alpha * np.dot(x.T, self.sigmoid(np.dot(x, weights)) - np.reshape(y, (len(y), 1)))
      cost_list[i] = cost(weights)
      self.weights = weights
      return cost_list

  def predict(self, x):
    z = np.dot(self.initialize(x)[1], self.weights)
    lis = []
    for i in self.sigmoid(z):
      if i > 0.5:
        lis.append(1)
      else:
        lis.append(0)
    return lis

In [8]:
obj = LogisticRegression()
model = obj.fit(X_train, Y_train)
y_pred = obj.predict(X_test)

In [9]:
from sklearn import metrics
print("Accuracy: ", metrics.accuracy_score(Y_test, y_pred))

Accuracy:  0.85


In [10]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

logreg.fit(X_train, Y_train)

y_prediction = logreg.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
from sklearn import metrics
print("Accuracy: ", metrics.accuracy_score(Y_test, y_prediction))

Accuracy:  0.85


##Logistic Regression Tensorflow

In [12]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import numpy as np

2 Loading and Preparing the MNIST Data Set

In [13]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Convert to float32.

x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

# Flatten images to 1-D vector of 784 features (28*28).
num_features=784
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])

# Normalize images value from [0, 255] to [0, 1].

x_train, x_test = x_train / 255., x_test / 255.

11501568/11490434 [==============================] - 0s 0us/step


3 Setting Up Hyperparameters and Data Set Parameters

In [14]:
# MNIST dataset parameters.
num_classes = 10 # 0 to 9 digits
num_features = 784 # 28*28
# Training parameters.
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50

4 Shuffling and Batching the Data

In [15]:
# Use tf.data API to shuffle and batch data.

train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

5 Initializing Weights and Biases

In [16]:
# Weight of shape [784, 10], the 28*28 image features, and a total number of classes.
W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")

# Bias of shape [10], the total number of classes.
b = tf.Variable(tf.zeros([num_classes]), name="bias")

6 Defining Logistic Regression and Cost Function

In [17]:
# Logistic regression (Wx + b).
def logistic_regression(x):
  # Apply softmax to normalize the logits to a probability distribution.
    return tf.nn.softmax(tf.matmul(x, W) + b)

# Cross-Entropy loss function
def cross_entropy(y_pred, y_true):
    # Encode label to a one hot vector.
    y_true = tf.one_hot(y_true, depth=num_classes)

    # Clip prediction values to avoid log(0) error.
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)

    # Compute cross-entropy.
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

7 Defining Optimizers and Accuracy Metrics

In [18]:
# Accuracy metric.
def accuracy(y_pred, y_true):

# Predicted class is the index of the highest score in prediction vector (i.e. argmax).
  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.SGD(learning_rate)

8 Optimization Process and Updating Weights and Biases

In [19]:
# Optimization process.
def run_optimization(x, y):

# Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)

    # Compute gradients.
    gradients = g.gradient(loss, [W, b])

  

    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, [W, b]))

9 The Training Loop

In [20]:
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):

    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)
    if step % display_step == 0:
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 50, loss: 594.708740, accuracy: 0.804688
step: 100, loss: 93.295120, accuracy: 0.902344
step: 150, loss: 177.833710, accuracy: 0.839844
step: 200, loss: 105.980347, accuracy: 0.902344
step: 250, loss: 85.613998, accuracy: 0.898438
step: 300, loss: 167.286224, accuracy: 0.832031
step: 350, loss: 50.125114, accuracy: 0.949219
step: 400, loss: 54.634823, accuracy: 0.945312
step: 450, loss: 62.807526, accuracy: 0.921875
step: 500, loss: 68.777725, accuracy: 0.898438
step: 550, loss: 52.058693, accuracy: 0.937500
step: 600, loss: 199.692764, accuracy: 0.781250
step: 650, loss: 63.757187, accuracy: 0.914062
step: 700, loss: 34.857971, accuracy: 0.953125
step: 750, loss: 322.942871, accuracy: 0.820312
step: 800, loss: 76.137573, accuracy: 0.925781
step: 850, loss: 71.589256, accuracy: 0.933594
step: 900, loss: 88.023056, accuracy: 0.894531
step: 950, loss: 88.434753, accuracy: 0.898438
step: 1000, loss: 45.094986, accuracy: 0.933594


10 Testing Model Accuracy Using the Test Data

In [21]:
# Test model on validation set.
pred = logistic_regression(x_test)
print("Test Accuracy: %f" % accuracy(pred, y_test))

Test Accuracy: 0.910400
